In [ ]:
import tensorflow as tf
from tensorflow import keras
import os
from tensorflow.keras.layers import ( Dense, Dropout,  LayerNormalization,)
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# For multiple devices (GPUs: 4, 5, 6, 7)
# os.environ["CUDA_VISIBLE_DEVICES"] = "1,4,5,6,7"
# import imageio
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import numpy as np
import math
import time, sys
import pickle
import timeit
from scipy.interpolate import make_interp_spline, BSpline
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras import layers
#import tensorflow_addons as tfa
from keras.optimizers import Adam
import keras.backend as K
import pandas as pd
#import wandb
#os.environ["WANDB_API_KEY"] = "key_code"

import numpy as np
#!pip install tensorflow_addons
import tensorflow as tf
from tensorflow import keras
import math
from tensorflow.keras import layers
#import tensorflow_addons as tfa
from keras.optimizers import Adam
import keras.backend as K
import pandas as pd
plt.ioff()

mnist = tf.keras.datasets.mnist
# update_progress() : Displays or updates a console progress bar
## Accepts a float between 0 and 1. Any int will be converted to a float.
## A value under 0 represents a 'halt'.
## A value at 1 or bigger represents 100%
def update_progress(progress):
    barLength = 10  # Modify this to change the length of the progress bar
    status = ""
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
        status = "error: progress var must be float\r\n"
    if progress < 0:
        progress = 0
        status = "Halt...\r\n"
    if progress >= 1:
        progress = 1
        status = "Done...\r\n"
    block = int(round(barLength * progress))
    text = "\rPercent: [{0}] {1}% {2}".format("#" * block + "-" * (barLength - block), progress * 100, status)
    sys.stdout.write(text)
    sys.stdout.flush()


# class Patches(tf.keras.layers.Layer):
#     def __init__(self, patch_size):
#         super(Patches, self).__init__()
#         self.patch_size = patch_size

#     def call(self, images):
#         batch_size = tf.shape(images)[0]
#         patches = tf.image.extract_patches(
#             images=images,
#             sizes=[1, self.patch_size, self.patch_size,  1],
#             strides=[1, self.patch_size, self.patch_size, 1],
#             rates=[1, 1,1, 1],
#             padding="VALID",
#         )
#         patch_dims = patches.shape[-1]
#         patches = tf.reshape(patches, [batch_size, -1, patch_dims])
#         return patches

# **Auxiliary Functions**

In [ ]:
def x_Sigma_w_x_T(x, W_Sigma):
  batch_sz = x.shape[0]
  xx_t = tf.reduce_sum(tf.multiply(x, x),axis=-1, keepdims=True)          #[50, 17, 64]  -> [50, 17, 1] or [50, 64] - > [50, 1]
 # xx_t_e = tf.expand_dims(xx_t,axis=2)                                   
  return tf.multiply(xx_t_e, W_Sigma) #[50,17,64] or [50, 64] or [50, 10]

def w_t_Sigma_i_w(w_mu, in_Sigma):  #[64, 64]  , [50, 17, 64] or [64, 10], [50, 64]
  Sigma_1 = tf.matmul(in_Sigma, tf.multiply(w_mu, w_mu))      #[50, 17, 64] or [50, 10]
  return Sigma_1

def tr_Sigma_w_Sigma_in(in_Sigma, W_Sigma):
  Sigma = tf.reduce_sum(in_Sigma, axis=-1, keepdims=True ) #[50,17, 1]
  return tf.multiply(Sigma, W_Sigma)  #[50,17, 64]


# def sigma_regularizer1(x):
#     input_size = 1.0   
#     f_s = tf.math.softplus(x) #tf.math.log(1. + tf.math.exp(x)) 
#     return  input_size * tf.reduce_mean(1. + tf.math.log(f_s) - f_s )

# **Bayesian Dense Layers**

In [ ]:
class LinearFirst(keras.layers.Layer):
    """y = w.x + b"""
    def __init__(self, units):
        super(LinearFirst, self).__init__()
        self.units = units
    def build(self, input_shape):
        tau = -1. /input_shape[-1]
        self.w_mu = self.add_weight(name='w_mu',
            shape=(input_shape[-1], self.units),
            initializer=tf.random_normal_initializer(mean=0.0, stddev=0.05, seed=None),regularizer=tf.keras.regularizers.l2(tau)
            trainable=True        )
        self.w_sigma = self.add_weight(name='w_sigma',
            shape=(self.units,),
            initializer=tf.random_uniform_initializer(minval= -12., maxval=-2.2, seed=None), regularizer=sigma_regularizer2,
            trainable=True        )

    def call(self, inputs):# [50,17,64]
        # Mean
        #print(self.w_mu.shape)
        mu_out = tf.matmul(inputs, self.w_mu) #+ self.b_mu       [50, 17, 64]             # Mean of the output
        # Varinace
        W_Sigma = tf.math.log(1. + tf.math.exp(self.w_sigma))         #[64]                        # Construct W_Sigma from w_sigmas
        Sigma_out = x_Sigma_w_x_T(inputs, W_Sigma)# [50, 17, 64]            + tf.math.log(1. + tf.math.exp(self.b_sigma)) #tf.linalg.diag(self.b_sigma)
        Sigma_out = tf.where(tf.math.is_nan(Sigma_out), tf.zeros_like(Sigma_out), Sigma_out)
        Sigma_out = tf.where(tf.math.is_inf(Sigma_out), tf.zeros_like(Sigma_out), Sigma_out)     
        Sigma_out = tf.abs(Sigma_out)      
        return mu_out, Sigma_out


class LinearNotFirst(keras.layers.Layer):
    """y = w.x + b"""
    def __init__(self, units):
        super(LinearNotFirst, self).__init__()
        self.units = units
                
    def build(self, input_shape):
        ini_sigma = -2.2
        #min_sigma = -4.5
        tau = -1. /input_shape[-1]    
        
        self.w_mu = self.add_weight(name = 'w_mu', shape=(input_shape[-1], self.units),   #[64 , 64] or or [64, 10] or [10, 10]
            initializer=tf.random_normal_initializer( mean=0.0, stddev=0.05, seed=None), regularizer=tf.keras.regularizers.l2(tau),#tau/self.units), #tf.keras.regularizers.l2(0.5*0.001),
            trainable=True,        )
        self.w_sigma = self.add_weight(name = 'w_sigma',
            shape=(self.units,),
            initializer= tf.constant_initializer(ini_sigma),#tf.random_uniform_initializer(minval= min_sigma, maxval=ini_sigma, seed=None) , 
            regularizer=sigma_regularizer2, #   tf.constant_initializer(ini_sigma)
            trainable=True,        )   
    def call(self, mu_in, Sigma_in):     # [50,17,64],  [50,17,64]   or [50, 64] or [50, 10]
        mu_out = tf.matmul(mu_in, self.w_mu) #+ self.b_mu  [50, 17, 64] 

        W_Sigma = tf.math.log(1. + tf.math.exp(self.w_sigma))    # [64] 
        Sigma_1 = w_t_Sigma_i_w (self.w_mu, Sigma_in) [50,17,64]
        Sigma_2 = x_Sigma_w_x_T(mu_in, W_Sigma)    # [50, 17, 64]                              
        Sigma_3 = tr_Sigma_w_Sigma_in (Sigma_in, W_Sigma) #[50, 17, 64]
        Sigma_out = Sigma_1 + Sigma_2 + Sigma_3 #+ tf.linalg.diag(tf.math.log(1. + tf.math.exp(self.b_sigma)))  #[50, 17, 64]
        
        Sigma_out = tf.where(tf.math.is_nan(Sigma_out), tf.zeros_like(Sigma_out), Sigma_out)  #[50,2,17,64,64]
        Sigma_out = tf.where(tf.math.is_inf(Sigma_out), tf.zeros_like(Sigma_out), Sigma_out)  #[50,2,17,64,64]
        Sigma_out = tf.abs(Sigma_out) 
        return mu_out, Sigma_out   # mu_out=[50,17,64], Sigma_out = [50,17,64] 

# **Bayesian Activation Functions**

In [ ]:
class VDP_GeLU(keras.layers.Layer):
    def __init__(self):
        super(VDP_GeLU, self).__init__()
    def call(self, mu_in,Sigma_in):   #mu_in = [50,17,64], Sigma_in= [50,17,64]
        mu_out = tf.nn.gelu(mu_in)  #[50,17,64]
        with tf.GradientTape() as g:
            g.watch(mu_in)
            out = tf.nn.gelu(mu_in) 
        gradi = g.gradient(out, mu_in) #[50,17,64]
        Sigma_out = activation_sigma(gradi, Sigma_in)
        return mu_out, Sigma_out  #[50,2,17,64], [50,2,17,64,64]


def activation_Sigma (gradi, Sigma_in):
  grad1 = tf.multiply(gradi,gradi) #[50,17,64] or [50, 10]
  return tf.multiply(Sigma_in, grad1) #[50,17,64] or [50, 10]


 class VDP_ReLU(keras.layers.Layer):
    """ReLU"""
    def __init__(self):
        super(VDP_ReLU, self).__init__()
    def call(self, mu_in, Sigma_in):
        mu_out = tf.nn.relu(mu_in)
        with tf.GradientTape() as g:
          g.watch(mu_in)
          out = tf.nn.relu(mu_in)
        gradi = g.gradient(out, mu_in) 
        Sigma_out = activation_Sigma(gradi, Sigma_in)        
        return mu_out, Sigma_out

# **Bayesian Dropout**

In [ ]:
class VDP_Dropout(keras.layers.Layer):
  def __init__(self, drop_prop):
    super(VDP_Dropout, self).__init__()
    self.drop_prop = drop_prop
  def call(self, mu_in,Sigma_in, Training=True):
    # shape=[batch_size, seq length, embedding_dim]
    scale_sigma = 1.0 / (1 - self.drop_prop)
    if Training:
      mu_out = tf.nn.dropout(mu_in, rate=self.drop_prop) #[50,17,64] or [50, 10]
      non_zero = tf.not_equal(mu_out, tf.zeros_like(mu_out))#[50,17,64]
      non_zero_sigma_mask = tf.boolean_mask(Sigma_in, non_zero) 
      idx_sigma = tf.dtypes.cast(tf.where(non_zero), tf.int32)
      Sigma_out =  (scale_sigma**2) * tf.scatter_nd(idx_sigma, non_zero_sigma_mask, tf.shape(non_zero)) 
    else:
      mu_out = mu_in
      Sigma_out = Sigma_in
    return mu_out,Sigma_out  #[50,17,64], [50,17,64]

# **Bayesian MLP**

In [ ]:
class VDP_MLP(tf.keras.layers.Layer):
    def __init__(self, hidden_features, out_features, dropout_rate=0.1):
        super(VDP_MLP, self).__init__()
        self.dense1 = LinearNotFirst(hidden_features)
        self.dense2 = LinearNotFirst(out_features)
        self.dropout1 =VDP_Dropout(dropout_rate)
        self.gelu_1 = VDP_GeLU()
    def call(self, mu_in, sigma_in):
        mu_out, sigma_out = self.dense1(mu_in, sigma_in)
    #    print('shape of x(MLP layer) :',x.shape)
        mu_out, sigma_out = self.gelu_1(mu_out, sigma_out)
     #   print('shape of x through GeLU :',x.shape)
        mu_out, sigma_out = self.dropout1(mu_out, sigma_out)
    #    print('shape of x after dropout :',x.shape)
        mu_out, sigma_out = self.dense2(mu_out, sigma_out)
    #    print('shape of x after 2nd dense :',x.shape)
        mu_out, sigma_out = self.dropout1(mu_out, sigma_out)
     #   print('shape of y', y.shape)
        return mu_out, sigma_out

# **Deterministic Layer Norm**

In [ ]:
class LayerNorm(tf.keras.layers.Layer):
    def __init__(self, eps=1e-6, **kwargs):
      self.eps = eps
      super(LayerNorm, self).__init__(**kwargs)

    def build(self, input_shape):
      self.gamma = self.add_weight(name='gamma', shape=input_shape[-1:],
                                   initializer=tf.keras.initializers.Ones(), trainable=True)
      self.beta = self.add_weight(name='beta', shape=input_shape[-1:],
                                  initializer=tf.keras.initializers.Zeros(), trainable=True)
      super(LayerNorm, self).build(input_shape)

    def call(self, x):
      mean = K.mean(x, axis=-1, keepdims=True)
      std = K.std(x, axis=-1, keepdims=True)
      #print( "mean of LN",mean.shape)
      #print("std of LN",std.shape)
      return self.gamma * (x - mean) / (std + self.eps) + self.beta

    def compute_output_shape(self, input_shape):
      return input_shape

# **Bayesian Layernorm**

In [ ]:
class Bayesian_LayerNorm(layers.Layer):

    def __init__(self, eps=1e-6, **kwargs):
        self.eps = eps
        super(Bayesian_LayerNorm, self).__init__(**kwargs)

    def build(self, input_shape):
        self.gamma = self.add_weight(name='gamma', shape=input_shape[-1:],
                                     initializer=tf.keras.initializers.Ones(), trainable=True)
        self.beta = self.add_weight(name='beta', shape=input_shape[-1:],
                                    initializer=tf.keras.initializers.Zeros(), trainable=True)
        super(Bayesian_LayerNorm, self).build(input_shape)
    def call(self, mu_x, sigma_x):  # (batch_size, sequence_length, embedding_dim),  (batch_size, sequence_length, embedding_dim)[50,17, 64],[50,17,64]
        mean = K.mean(mu_x, axis=-1, keepdims=True) #[50,17,1]
        std = K.std(mu_x, axis=-1, keepdims=True) #[50,17,1]
       # print('std = ' , std.shape)
      #  print('gamma = ',self.gamma)
        out_mu= self.gamma * (mu_x - mean) / (std + self.eps) + self.beta
        a = (self.gamma/(std+self.eps))**2 #[50,17,64]
  
        out_sigma = tf.math.multiply(a , sigma_x)   #[50,17,64]
        return out_mu, out_sigma

    def compute_output_shape(self, input_shape):
        return input_shape

# **Bayesian Multi-Head Attention**

In [ ]:
class Bayesian_MultiHeadSelfAttention_First(tf.keras.layers.Layer):
  def __init__(self, embed_dim, num_heads=8):
    super(Bayesian_MultiHeadSelfAttention_First, self).__init__()
    self.embed_dim = embed_dim
    self.num_heads = num_heads
    if embed_dim % num_heads != 0:
      raise ValueError(
        f"embedding dimension = {embed_dim} should be divisible by number of heads = {num_heads}"
      )
    self.projection_dim = embed_dim // num_heads
    self.query_dense = LinearFirst(embed_dim)
    self.key_dense = LinearFirst(embed_dim)
    self.value_dense = LinearFirst(embed_dim)
    self.combine_heads = LinearFirst(embed_dim)

  def attention(self, mu_query,sigma_query, mu_key,sigma_key, mu_value,sigma_value, input_dimension):
    mu_score = tf.matmul(mu_query, mu_key, transpose_b=True) # [50, 2, 17, 32] x [50, 2, 32, 17] = [50, 2, 17, 17]

    a = tf.reduce_sum(tf.math.multiply(mu_query**2 , sigma_key), axis=-1, keepdims=True) # [50, 2, 17, 1] 
    b = tf.transpose(tf.reduce_sum(tf.math.multiply(mu_key**2 , sigma_query), axis=-1, keepdims=True), perm=[0, 1, 3, 2])# [50, 2, 1, 17 ]
    a_b= a + b # [50, 2, 17, 17] 

    
    c1 = tf.reduce_sum(tf.math.multiply(sigma_query, sigma_key), axis=-1, keepdims=True) # [50, 2, 17, 1]
    c2 = tf.transpose(c1, perm=[0, 1, 3, 2] ) # [50, 2, 1, 17]
    c = c1 + c2 # [50, 2, 17, 17]     
    sigma_score = a_b + c # [50, 2, 17, 17]
    dim_key = tf.cast(tf.shape(mu_key)[-1], tf.float32)
    mu_scaled_score = mu_score / tf.math.sqrt(dim_key) #[50, 2, 17, 17]
    sigma_scaled_score = sigma_score * dim_key # [50, 2, 17, 17]

    mu_weights = tf.nn.softmax(mu_scaled_score, axis=-1) # [50, 2, 17, 17]
    # Sigma for softmax function
    pp1 = tf.expand_dims(mu_weights, axis=-1) # [50, 2, 17, 17,1]
    pp2 = tf.expand_dims(mu_weights, axis=3) # [50, 2, 17,1, 17]
    ppT = tf.matmul(pp1, pp2) # # [50, 2, 17, 17,17]
    p_diag = tf.linalg.diag(mu_weights) # [50, 2, 17, 17,17]
    grad = (p_diag - ppT)**2 # # [50, 2, 17, 17,17]
    Sigma_weights = tf.squeeze(tf.matmul(grad, tf.expand_dims(sigma_scaled_score, axis=-1))) # [50, 2, 17, 17]
    Sigma_weights = tf.where(tf.math.is_nan(Sigma_out), tf.zeros_like(Sigma_out), Sigma_ou
    Sigma_weights = tf.where(tf.math.is_inf(Sigma_out), tf.zeros_like(Sigma_out), Sigma_out)
   # Sigma_weights = tf.linalg.set_diag(Sigma_out, tf.abs(tf.linalg.diag_part(Sigma_out)))

    mu_output = tf.matmul(mu_weights, mu_value)  #[50,2,17,17] X [50,2,17,32]=  [50,2,17,32] 
    d = tf.matmul(mu_weights**2,sigma_value)) #[50,2,17,32] 
    e = tf.matmul(Sigma_weights, mu_value**2)) #[50,2,17,32] 
    f = tf.matmul(Sigma_weights, sigma_value) # [50, 2, 17, 17]x[50,2,17,32]=  [50,2,17,32] 
    output_sigma = d + e + f     
    return mu_output, sigma_output #, mu_weights, Sigma_weights

  def separate_heads(self, mu_x, sigma_x batch_size): # [50, 17,64], [50, 17, 64]
    mu_x = tf.reshape(mu_x, (batch_size, -1, self.num_heads, self.projection_dim)) # [50, 17, 2 ,32]
    sigma_x = tf.reshape(sigma_x, (batch_size, -1, self.num_heads, self.projection_dim)) # [50, 17, 2 32]
    mu_x = tf.transpose(mu_x, perm=[0, 2, 1, 3]) # [50, 2, 17, 32]
    sigma_x = tf.transpose(sigma_x, perm=[0, 2, 1, 3]) # [50, 2, 17, 32]
    return mu_x, sigma_x #[50,2,17,32],[50,2,17,32]

  def call(self, inputs):
    batch_size = tf.shape(inputs)[0]
    mu_query, sigma_query = self.query_dense(inputs)  # [50, 17,64]  , [50, 17,64] 
    mu_key, sigma_key = self.key_dense(inputs)   # [50, 17,64] , [50, 17,64] 
    mu_value, sigma_value = self.value_dense(inputs) # [50, 17,64], [50, 17,64] 

    mu_query,sigma_query = self.separate_heads(mu_query,sigma_query, batch_size)
    mu_key, sigma_key = self.separate_heads(mu_key, sigma_key, batch_size)
    mu_value,sigma_value = self.separate_heads(mu_value,sigma_value, batch_size)

    mu_attention, sigma_attention = self.attention(mu_query,sigma_query, mu_key, sigma_key, mu_value,sigma_value, tf.shape(inputs)[1])
    mu_attention = tf.transpose(mu_attention, perm=[0, 2, 1, 3])
    sigma_attention = tf.transpose(sigma_attention, perm=[0, 2, 1, 3])

    mu_concat_attention = tf.reshape(mu_attention, (batch_size, -1, self.embed_dim))
    sigma_concat_attention = tf.reshape(sigma_attention, (batch_size, -1, self.embed_dim))

    mu_output = self.combine_heads(mu_concat_attention)
    sigma_output = self.combine_heads(sigma_concat_attention)
    return mu_output, sigma_output

# **Bayesian Transformer Block**

In [ ]:
class VDP_TransformerBlock_first(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, mlp_dim, dropout=0.1):
        super(VDP_TransformerBlock_first, self).__init__()
        self.att = Bayesian_MultiHeadSelfAttention_First(embed_dim, num_heads)  #[64,2]
        self.mlp = VDP_MLP(mlp_dim*2,mlp_dim,dropout) #[64*2,64,dropout]
        self.layernorm1 = LayerNorm(eps=1e-6)
        self.layernorm2 = Bayesian_LayerNorm(eps=1e-6)
        self.dropout1 = VDP_Dropout(dropout)

    def call(self, inputs, training):
        inputs_norm = self.layernorm1(inputs) #[50,17,64]
        mu_output , sigma_out = self.att(inputs_norm) #[50,17,64]
        mu_output , sigma_out1 = self.dropout1(mu_output , sigma_out, training=training)#[50,17,64]
        mu_out1 = mu_output + inputs  #[50,17,64]

        mu_out1_norm, sigma_out1_norm = self.layernorm2(mu_out1, sigma_out1)
        mu_mlp_output, sigma_mlp_output = self.mlp(mu_out1_norm, sigma_out1_norm)
        mu_mlp_output, sigma_mlp_output = self.dropout1(mu_mlp_output, sigma_mlp_output, training=training)
        mu_output = mu_mlp_output + mu_out1
        with tf.GradientTape() as g:
             g.watch(mu_out1)
             out = mu_mlp_output + mu_out1
        gradi = g.gradient(out, mu_out1) 
        sigma_output = tf.math.multiply(tf.math.multiply(gradi, gradi), sigma_out1)
        return mu_output, sigma_output

# **Bayesian Transformer using Uncertainty Propagation Framework**

In [ ]:
class VDP_ViT(tf.keras.Model):
  def __init__(
        self,
        image_size,
        patch_size,
        num_layers,
        num_classes,
        d_model,
        num_heads,
        mlp_dim,
        channels=1,
        dropout=0.1,
    ):
      super(VDP_ViT, self).__init__()
      num_patches = (image_size // patch_size) ** 2
      self.patch_dim = channels * (patch_size ** 2)

      self.patch_size = patch_size
      self.d_model = d_model
      self.num_layers = num_layers

      self.rescale = Rescaling(1.0 / 255)
      self.pos_emb = self.add_weight(
        "pos_emb", shape=(1, num_patches + 1, d_model)
        )
      self.class_emb = self.add_weight("class_emb", shape=(1, 1, d_model))
      self.patch_proj = DDense(d_model)
      self.enc_layers = VDP_TransformerBlock_first(d_model, num_heads, mlp_dim, dropout))
     # self.enc_layers = [ VDP_TransformerBlock(d_model, num_heads, mlp_dim, dropout)
     #   for _ in range(num_layers)
     #   ]
      self.mlp_head =VDP_MLP(mlp_dim, num_classes)
  def extract_patches(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1,self.patch_size, self.patch_size, 1],
            strides=[1,self.patch_size, self.patch_size, 1],
            rates=[1,1, 1, 1],
            padding="VALID",
        )
        patches = tf.reshape(patches, [batch_size, -1, self.patch_dim])
        return patches

  def call(self, x, training):
        print('Input dimension :',x.shape)
        batch_size = tf.shape(x)[0]
        x = self.rescale(x)
        print('Input dimension after rescale :',x.shape)
        patches = self.extract_patches(x)
        print('Input dimension after extract patch :',patches.shape)
        x = self.patch_proj(patches)
        print('Input dimension after patch projection :',x.shape)
        class_emb = tf.broadcast_to(
        self.class_emb, [batch_size, 1, self.d_model]
        )
        x = tf.concat([class_emb, x], axis=1)
        print('Input dimension after concat :',x.shape)
        x = x + self.pos_emb
        print('Input dimension after x = x + self.pos_emb :',x.shape)
        mu_out, sigma_out = self.enc_layers(x)
   #     for layer in self.enc_layers:
   #         x = layer(x, training)

    # First (class token) is used for classification
        mu, sigma = self.mlp_head(mu_out[:, 0], sigma_out[:, 0] )
        return mu, sigma

# **Loss Function for the Bayesian Framework**

In [ ]:
def main_function(image_size=28,patch_size=7,num_layers=2,num_classes=10,d_model=64,num_heads=2,mlp_dim=64,channels=1,
                  dropout=0.1,
                  batch_size=50, epochs=10, lr=0.001, lr_end = 0.0001,
                  Targeted=False,
                Training=True, continue_training=False, saved_model_epochs=10):
    #PATH = './saved_models/cnn_epoch_{}/'.format(epochs)
    # the data, split between train and test sets
    (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

    # Scale images to the [0, 1] range
    x_train = x_train.astype("float32") / 255
    x_test = x_test.astype("float32") / 255

    # Make sure images have shape (28, 28, 1)
    x_train = np.expand_dims(x_train, -1)
    x_test = np.expand_dims(x_test, -1)

    trans_model = VDP_ViT(image_size=image_size, patch_size=patch_size, num_layers=num_layers,
                                  num_classes=num_classes, d_model= d_model, num_heads=num_heads,mlp_dim = mlp_dim,
                                  channels= channels)

    y_train = keras.utils.to_categorical(y_train, 10)
    y_test = keras.utils.to_categorical(y_test, 10)
    tr_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)
    val_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

    num_train_steps = epochs * int(x_train.shape[0] / batch_size)
    #    step = min(step, decay_steps)
    #    ((initial_learning_rate - end_learning_rate) * (1 - step / decay_steps) ^ (power) ) + end_learning_rate
    learning_rate_fn = tf.keras.optimizers.schedules.PolynomialDecay(initial_learning_rate=lr,
                                                                     decay_steps=num_train_steps,
                                                                     end_learning_rate=lr_end, power=2.)
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_fn)  # , clipnorm=1.0)
    loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    

    @tf.function  # Make it fast.
    def train_on_batch(x, y):
        with tf.GradientTape() as tape:
            trans_model.trainable = True
            out = trans_model(x, training=True)
            loss = loss_fn(y, out)
            gradients = tape.gradient(loss, trans_model.trainable_weights)

            #  gradients = [(tf.where(tf.math.is_nan(grad), tf.constant(1.0e-5, shape=grad.shape), grad)) for grad in gradients]
        #  gradients = [(tf.where(tf.math.is_inf(grad), tf.constant(1.0e-5, shape=grad.shape), grad)) for grad in gradients]
        optimizer.apply_gradients(zip(gradients, trans_model.trainable_weights))
        
        return loss, out

    @tf.function
    def validation_on_batch(x, y):
        trans_model.trainable = False
        out = trans_model(x, training=False)
        total_vloss = loss_fn(y, out)
        return total_vloss, out

    @tf.function
    def test_on_batch(x, y):
        trans_model.trainable = False
        out = trans_model(x, training=False)
        return out
    if Training: 
       # wandb.init(entity = "dimah", project="DCNN_Cifar10_11layers_epochs_{}_lr_{}_latest".format(epochs, lr)) 
        
        if continue_training:
            saved_model_path = './saved_models/cnn_epoch_{}/'.format(saved_model_epochs)
            trans_model.load_weights(saved_model_path + 'Deterministic_cnn_model')
        
        train_acc = np.zeros(epochs)
        valid_acc = np.zeros(epochs)        
        train_err = np.zeros(epochs)
        valid_err = np.zeros(epochs)                      
        start = timeit.default_timer()
        for epoch in range(epochs):
            print('Epoch: ', epoch + 1, '/', epochs)            
            tr_no_steps = 0
            va_no_steps = 0
            # -------------Training--------------------
            acc_training = np.zeros(int(x_train.shape[0] / (batch_size)))
            err_training = np.zeros(int(x_train.shape[0] / (batch_size)))            
            for step, (x, y) in enumerate(tr_dataset):
                update_progress(step / int(x_train.shape[0] / (batch_size)))
                loss, out = train_on_batch(x, y)             
                err_training[tr_no_steps] = loss.numpy()              
                corr = tf.equal(tf.math.argmax(out, axis=-1), tf.math.argmax(y, axis=-1))
                accuracy = tf.reduce_mean(tf.cast(corr, tf.float32))                  
                acc_training[tr_no_steps] = accuracy.numpy()                                               
                tr_no_steps += 1                
              
            train_acc[epoch] = np.mean(np.amax(acc_training))
            train_err[epoch] = np.mean(np.amin(err_training))
            print('Training Acc  ', train_acc[epoch])
            print('Training error', train_err[epoch])       
            # ---------------Validation----------------------  
            acc_validation = np.zeros(int(x_test.shape[0] / (batch_size)))
            err_validation = np.zeros(int(x_test.shape[0] / (batch_size)))                     
            for step, (x, y) in enumerate(val_dataset):
                update_progress(step / int(x_test.shape[0] / (batch_size)))
                total_vloss, out = validation_on_batch(x, y)                   
                err_validation[va_no_steps] = total_vloss.numpy()              
                corr = tf.equal(tf.math.argmax(out, axis=-1), tf.math.argmax(y, axis=-1))
                va_accuracy = tf.reduce_mean(tf.cast(corr, tf.float32))                 
                acc_validation[va_no_steps] = va_accuracy.numpy()                
                va_no_steps += 1               
            
            valid_acc[epoch] = np.mean(np.amax(acc_validation))
            valid_err[epoch] = np.mean(np.amin(err_validation))           
            stop = timeit.default_timer() 
            #cnn_model.save_weights(PATH + 'Deterministic_cnn_model')                   
##            wandb.log({"Training Loss":  train_err[epoch],                        
##                       "Training Accuracy": train_acc[epoch],                                             
##                        "Validation Loss": valid_err[epoch],                        
##                        "Validation Accuracy": valid_acc[epoch],                       
##                        'epoch': epoch
##                       })             
            print('Total Training Time: ', stop - start)
            print(' Training Acc   ', train_acc[epoch])            
            print(' Validation Acc ', valid_acc[epoch])            
            print('------------------------------------')
            print('Training error   ', train_err[epoch])            
            print('Validation error', valid_err[epoch])                    
            # -----------------End Training--------------------------                               
           
       
if __name__ == '__main__':
    main_function()

